# PyBroker Tutorial: Getting Started with Algorithmic Trading

## Introduction

Welcome to this tutorial on PyBroker, a powerful Python framework for developing algorithmic trading strategies. In this notebook, we'll walk through the process of installing PyBroker and getting started with some basic concepts.

## 1. Installation

First, let's install PyBroker. We'll use pip to install the latest version.

In [3]:
! pip install -U lib-pybroker


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import pybroker
print(f"PyBroker version: {pybroker.__version__}")

PyBroker version: 1.2.0


## 2. Data Sources
- yfinance
- akshare

caching data
- caching by calling: pybroker.enable_data_source_cache(‘name’) 
- clear cache using: pybroker.clear_data_source_cache()
- disable caching using: pybroker.disable_data_source_cache()

In [8]:
import pybroker
pybroker.enable_data_source_cache('yfinance')

In [5]:
from pybroker import YFinance

yfinance = YFinance()
df = yfinance.query(['AAPL', 'MSFT'], start_date='1/1/2021', end_date='1/1/2024')
df

Loading bar data...


[*********************100%%**********************]  2 of 2 completed

Loaded bar data: 0:00:06 



,date,symbol,open,high,low,close,volume,adj_close
0,2021-03-01,AAPL,123.750000,127.930000,122.790001,127.790001,116307900,125.429527
1,2021-03-01,MSFT,235.899994,237.470001,233.149994,236.940002,25324000,230.432037
2,2021-03-02,AAPL,128.410004,128.720001,125.010002,125.120003,102260900,122.808853
3,2021-03-02,MSFT,237.009995,237.300003,233.449997,233.869995,22812500,227.446396
4,2021-03-03,AAPL,124.809998,125.709999,121.839996,122.059998,112966300,119.805336
...,...,...,...,...,...,...,...,...
501,2022-02-24,MSFT,272.510010,295.160004,271.519989,294.589996,56989700,288.832275
502,2022-02-25,AAPL,163.839996,165.119995,160.869995,164.850006,91974200,162.766663
503,2022-02-25,MSFT,295.140015,297.630005,291.649994,297.309998,32546700,291.499054
504,2022-02-28,AAPL,163.059998,165.419998,162.429993,165.119995,95056600,163.033234


In [7]:
from pybroker.ext.data import AKShare

akshare = AKShare()
# You can substitute 000001.SZ with 000001, and it will still work!
# and you can set start_date as "20210301" format
# You can also set adjust to 'qfq' or 'hfq' to adjust the data,
# and set timeframe to '1d', '1w' to get daily, weekly data
df = akshare.query(
    symbols=['000001.SZ', '600000.SH'],
    start_date='1/1/2021',
    end_date='1/1/2024',
    adjust="qfq",
    timeframe="1d",
)
df

Loading bar data...
Loaded bar data: 0:00:15 



,date,symbol,open,high,low,close,volume
0,2021-03-01,000001.SZ,20.13,20.27,19.77,20.04,1125387
1,2021-03-01,600000.SH,9.38,9.43,9.29,9.37,547461
2,2021-03-02,000001.SZ,20.21,20.74,19.85,20.24,1473425
3,2021-03-02,600000.SH,9.40,9.49,9.15,9.26,747631
4,2021-03-03,000001.SZ,20.17,21.67,20.05,21.60,1919635
...,...,...,...,...,...,...,...
969,2023-02-27,600000.SH,6.84,6.88,6.84,6.84,158006
970,2023-02-28,000001.SZ,12.75,12.85,12.61,12.78,607936
971,2023-02-28,600000.SH,6.86,6.88,6.82,6.86,174481
972,2023-03-01,000001.SZ,12.80,13.19,12.74,13.17,1223452


## 3. Backtesting a Strategy
 set up a new instance of the Strategy class which will be used to perform a backtest on the trading strategy

In [9]:
import pybroker
from pybroker import Strategy, StrategyConfig, YFinance

pybroker.enable_data_source_cache('my_strategy')

config = StrategyConfig(initial_cash=500_000)
strategy = Strategy(YFinance(), '1/1/2021', '1/1/2024', config)

In [10]:
def buy_low(ctx):
    # If shares were already purchased and are currently being held, then return.
    if ctx.long_pos():
        return
    # If the latest close price is less than the previous day's low price,
    # then place a buy order.
    if ctx.bars >= 2 and ctx.close[-1] < ctx.low[-2]:
        # Buy a number of shares that is equal to 25% the portfolio.
        ctx.buy_shares = ctx.calc_target_shares(0.25)
        # Set the limit price of the order.
        ctx.buy_limit_price = ctx.close[-1] - 0.01
        # Hold the position for 3 bars before liquidating (in this case, 3 days).
        ctx.hold_bars = 3

strategy.add_execution(buy_low, ['AAPL', 'MSFT'])

In [ ]:
def short_high(ctx):
    # If shares were already shorted then return.
    if ctx.short_pos():
        return
    # If the latest close price is more than the previous day's high price,
    # then place a sell order.
    if ctx.bars >= 2 and ctx.close[-1] > ctx.high[-2]:
        # Short 100 shares.
        ctx.sell_shares = 100
        # Cover the shares after 2 bars (in this case, 2 days).
        ctx.hold_bars = 2
strategy.add_execution(short_high, ['TSLA'])

In [15]:
result = strategy.backtest()

Backtesting: 2021-01-01 00:00:00 to 2024-01-01 00:00:00

Loaded cached bar data.

Test split: 2021-01-04 00:00:00 to 2023-12-29 00:00:00


  0% (0 of 753) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
 26% (201 of 753) |#####                 | Elapsed Time: 0:00:00 ETA:   0:00:00
 57% (431 of 753) |############          | Elapsed Time: 0:00:00 ETA:   0:00:00
 87% (661 of 753) |###################   | Elapsed Time: 0:00:00 ETA:   0:00:00
100% (753 of 753) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


In [ ]:
import matplotlib.pyplot as plt

chart = plt.subplot2grid((3, 2), (0, 0), rowspan=3, colspan=2)
chart.plot(result.portfolio.index, result.portfolio['market_value'])

In [20]:
result.positions

long_shares  short_shares   close     equity  market_value  \
symbol date                                                                     
TSLA   2021-01-07            0           100  272.01       0.00      26536.00   
       2021-01-08            0           100  293.34       0.00      26536.00   
AAPL   2021-01-12          967             0  128.80  124549.60     124549.60   
       2021-01-13          967             0  130.89  126570.63     126570.63   
       2021-01-14          967             0  128.91  124655.97     124655.97   
...                        ...           ...     ...        ...           ...   
       2023-12-22          732             0  193.60  141715.20     141715.20   
       2023-12-26          732             0  193.05  141312.60     141312.60   
       2023-12-27          732             0  193.15  141385.80     141385.80   
TSLA   2023-12-28            0           100  253.18       0.00      25892.00   
       2023-12-29            0           100  248.48       0.00      25892.00   

                     margin  unrealized_pnl  
symbol date                                  
TSLA   2021-01-07  27201.33         -665.33  
       2021-01-08  29334.00        -2798.00  
AAPL   2021-01-12      0.00          502.84  
       2021-01-13      0.00         2523.87  
       2021-01-14      0.00          609.21  
...                     ...             ...  
       2023-12-22      0.00         -431.88  
       2023-12-26      0.00         -834.48  
       2023-12-27      0.00         -761.28  
TSLA   2023-12-28  25318.00          574.00  
       2023-12-29  24848.00         1044.00  

[609 rows x 7 columns]

In [21]:
result.trades

,type,symbol,entry_date,exit_date,entry,exit,shares,pnl,return_pct,agg_pnl,bars,pnl_per_bar,stop,mae,mfe
id,,,,,,,,,,,,,,,
1,short,TSLA,2021-01-07,2021-01-11,265.36,276.34,100,-1098.00,-3.97,-1098.00,2,-549.00,bar,-29.47,6.96
2,long,AAPL,2021-01-12,2021-01-15,128.28,128.61,967,319.11,0.26,-778.89,3,106.37,bar,-1.42,3.17
3,short,TSLA,2021-01-21,2021-01-25,282.86,289.87,100,-701.00,-2.42,-1479.89,2,-350.50,bar,-7.01,6.65
4,short,TSLA,2021-01-26,2021-01-28,294.58,274.83,100,1975.00,7.19,495.11,2,987.50,bar,-4.05,19.75
5,long,AAPL,2021-01-29,2021-02-03,133.48,134.69,912,1103.52,0.91,1598.63,3,367.84,bar,-3.27,3.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,long,AAPL,2023-11-27,2023-11-30,189.78,189.26,748,-388.96,-0.27,68793.03,3,-129.65,bar,-0.88,2.31
242,short,TSLA,2023-11-29,2023-12-01,247.75,236.04,100,1171.00,4.96,69964.03,2,585.50,bar,-5.00,11.71
243,long,MSFT,2023-12-04,2023-12-07,366.21,368.89,380,1018.40,0.73,70982.43,3,339.47,bar,-3.31,7.97


In [22]:
result.orders

,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,sell,TSLA,2021-01-07,100,NaN,265.36,0.0
2,buy,TSLA,2021-01-11,100,NaN,276.34,0.0
3,buy,AAPL,2021-01-12,967,128.97,128.28,0.0
4,sell,AAPL,2021-01-15,967,NaN,128.61,0.0
5,sell,TSLA,2021-01-21,100,NaN,282.86,0.0
...,...,...,...,...,...,...,...
487,sell,TSLA,2023-12-14,100,NaN,247.33,0.0
488,buy,TSLA,2023-12-18,100,NaN,255.05,0.0
489,buy,AAPL,2023-12-22,732,194.67,194.19,0.0


In [23]:
result.metrics_df

,name,value
0,trade_count,245.000000
1,initial_market_value,500000.000000
2,end_market_value,571056.790000
3,total_pnl,70012.790000
4,unrealized_pnl,1044.000000
5,total_return_pct,14.002558
6,total_profit,265724.120000
7,total_loss,-195711.330000
8,total_fees,0.000000
9,max_drawdown,-29469.280000


## 4. Evaluating
Bootstrap metrics can help us to more thoroughly evaluate a trading strategy

In [24]:
import pybroker
from pybroker import Strategy, StrategyConfig, YFinance

pybroker.enable_data_source_cache('my_strategy')

def buy_low(ctx):
    if ctx.long_pos():
        return
    if ctx.bars >= 2 and ctx.close[-1] < ctx.low[-2]:
        ctx.buy_shares = ctx.calc_target_shares(0.25)
        ctx.buy_limit_price = ctx.close[-1] - 0.01
        ctx.hold_bars = 3

def short_high(ctx):
    if ctx.short_pos():
        return
    if ctx.bars >= 2 and ctx.close[-1] > ctx.high[-2]:
        ctx.sell_shares = 100
        ctx.hold_bars = 2

config = StrategyConfig(initial_cash=500_000, bootstrap_sample_size=100)
strategy = Strategy(YFinance(), '3/1/2017', '3/1/2022', config)
strategy.add_execution(buy_low, ['AAPL', 'MSFT'])
strategy.add_execution(short_high, ['TSLA'])

result = strategy.backtest(calc_bootstrap=True)
result.metrics_df

Backtesting: 2017-03-01 00:00:00 to 2022-03-01 00:00:00

Loading bar data...


[*********************100%%**********************]  3 of 3 completed

Loaded bar data: 0:00:01 

Test split: 2017-03-01 00:00:00 to 2022-02-28 00:00:00



  0% (0 of 1259) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--
 19% (241 of 1259) |####                 | Elapsed Time: 0:00:00 ETA:   0:00:00
 39% (501 of 1259) |########             | Elapsed Time: 0:00:00 ETA:   0:00:00
 61% (771 of 1259) |############         | Elapsed Time: 0:00:00 ETA:   0:00:00
 82% (1041 of 1259) |################    | Elapsed Time: 0:00:00 ETA:   0:00:00
100% (1259 of 1259) |####################| Elapsed Time: 0:00:00 Time:  0:00:00



Calculating bootstrap metrics: sample_size=100, samples=10000...
Calculated bootstrap metrics: 0:00:02 

Finished backtest: 0:00:04


,name,value
0,trade_count,388.000000
1,initial_market_value,500000.000000
2,end_market_value,665009.260000
3,total_pnl,165830.590000
4,unrealized_pnl,-821.330000
5,total_return_pct,33.166118
6,total_profit,402053.210000
7,total_loss,-236222.620000
8,total_fees,0.000000
9,max_drawdown,-31619.460000


In [25]:
result.bootstrap.conf_intervals

lower     upper
name          conf                     
Profit Factor 97.5%  0.572351  4.089495
              95%    0.686388  3.537733
              90%    0.846766  3.009238
Sharpe Ratio  97.5% -0.148176  0.239946
              95%   -0.110546  0.214665
              90%   -0.069284  0.186993

In [26]:
result.bootstrap.drawdown_conf

,amount,percent
conf,,
99.9%,-66856.42,-10.765212
99%,-50523.82,-8.102360
95%,-38047.80,-6.126216
90%,-31668.81,-5.171825


## 5. Training a Model

train and backtest machine learning models
- accurate predictions about market movements
- https://www.pybroker.com/en/latest/notebooks/6.%20Training%20a%20Model.html

---

In [3]:
# simple moving average crossover strategy  
from pybroker import Strategy, YFinance
from pybroker.indicators import sma

def ma_crossover_strategy(ctx):
    fast_ma = ctx.indicator('fast_ma')
    slow_ma = ctx.indicator('slow_ma')
    
    if fast_ma[-1] > slow_ma[-1] and fast_ma[-2] <= slow_ma[-2]:
        ctx.buy_shares = 100
    elif fast_ma[-1] < slow_ma[-1] and fast_ma[-2] >= slow_ma[-2]:
        ctx.sell_all_shares()

# Create a strategy using Yahoo Finance data
strategy = Strategy(YFinance(), start_date='2022-01-01', end_date='2023-01-01')

# Add our execution function to the strategy
strategy.add_execution(
    ma_crossover_strategy, 
    ['AAPL'], 
    indicators=[
        sma('fast_ma', 'close', 10),
        sma('slow_ma', 'close', 30)
    ]
)

# Run the backtest
result = strategy.backtest()

# Print some basic results
print(f"Total Return: {result.total_return:.2%}")
print(f"Sharpe Ratio: {result.sharpe_ratio:.2f}")

ImportError: cannot import name 'sma' from 'pybroker.indicator' (/Users/liang/miniconda3/lib/python3.10/site-packages/pybroker/indicator.py)